#### Notes on current version:
recentered_input vs norm_at_center?

# NEU (Reconfigurations Map and Related Functions)

### Basic Algorithm (NEU-OLS)

1. Perform Basic Algorithm (in this case OLS)
2. Map predictions to their graph; ie $x\mapsto (x,\hat{f}_{OLS}(x))$ where $\hat{f}_{OLS}$ is the least-squares regression function.

## Initializations:

In [1]:
# Deep Learning & ML
import tensorflow as tf
import tensorflow_probability as tfp
import keras as K
import keras.backend as Kb
from keras.layers import *

from keras.models import Model
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils
from scipy import linalg as scila

from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.constraints import NonNeg



# Linear Regression
from sklearn.linear_model import LinearRegression

# General
import numpy as np
import time

# Alerts
import os as beepsnd

# Others
import math

# General Outputs
print('TensorFlow:', tf.__version__)

Using TensorFlow backend.


TensorFlow: 2.1.0


In [2]:
N_Reconfigurations = 10**2
d = 1 # Dimension of X
D = 1 # Dimension of Y


# Data Meta-Parameters
noise_level = 0.1
uncertainty_level= 0.5

# Training meta-parameters
Pre_Epochs = 50
Full_Epochs = 100

# # Height Per Reconfiguration
# Height_factor_Per_reconfig = d+D

# Number of Datapoints
N_data = 10**3
# Unknown Function
def unknown_f(x):
    return np.sin(x) #+ (x % 2)

# Generate Data
%run Data_Generator.ipynb

<Figure size 640x480 with 1 Axes>

#### Prepare data for NEU

In [3]:
# Reshape Data Into Compatible Shape
data_x = np.array(data_x).reshape(-1,d)
data_y = np.array(data_y)
# Perform OLS Regression
linear_model = LinearRegression()
reg = linear_model.fit(data_x, data_y)
model_pred_y = linear_model.predict(data_x)
# Map to Graph
data_NEU = np.concatenate((data_x,model_pred_y.reshape(-1,D)),1)
NEU_targets  = data_y.reshape(-1,D)

### Build Reconfiguration Unit
$$
x \mapsto \exp\left(
%\psi(a\|x\|+b)
\operatorname{Skew}_d\left(
    F(\|x\|)
\right)
\right) x.
$$

In [4]:
class Reconfiguration_unit(tf.keras.layers.Layer):
    
    def __init__(self, units=16, input_dim=32):
        super(Reconfiguration_unit, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        #------------------------------------------------------------------------------------#
        # Center
        #------------------------------------------------------------------------------------#
        self.location = self.add_weight(name='location',
                                    shape=(self.units,),
                                    initializer='random_normal',
                                    trainable=True)
        
        #------------------------------------------------------------------------------------#
        # Bump Function
        #------------------------------------------------------------------------------------#
#         self.sigma = self.add_weight(name='bump_threshfold',
#                                         shape=[1],
#                                         initializer=RandomUniform(minval=.5, maxval=1),
#                                         trainable=True,
#                                         constraint=tf.keras.constraints.NonNeg())
#         self.a = self.add_weight(name='bump_scale',
#                                         shape=[1],
#                                         initializer='ones',
#                                         trainable=True)
#         self.b = self.add_weight(name='bump_location',
#                                         shape=[1],
#                                         initializer='zeros',
#                                         trainable=True)

        #------------------------------------------------------------------------------------#
        # Tangential Map
        #------------------------------------------------------------------------------------#
        self.Tw1 = self.add_weight(name='Tangential_Weights_1 ',shape=(input_shape[-1],input_shape[-1]),initializer='GlorotUniform',trainable=True)
        
        self.Tw2 = self.add_weight(name='Tangential_Weights_2 ',shape=(input_shape[-1],input_shape[-1]),initializer='GlorotUniform',trainable=True)

        self.Tb1 = self.add_weight(name='Tangential_basies_1',shape=(input_shape[-1],input_shape[-1]),initializer='GlorotUniform',trainable=True)
        self.num_stab_param = self.add_weight(name='matrix_exponential_stabilizer',shape=[1],initializer=RandomUniform(minval=0.0, maxval=0.01),trainable=True,constraint=tf.keras.constraints.NonNeg())
        
        # Wrap things up!
        super().build(input_shape)

    def bump_function(self, x):
        return tf.math.exp(-self.sigma / (self.sigma - x))

        
    def call(self, input):
        #------------------------------------------------------------------------------------#
        # Initializations
        #------------------------------------------------------------------------------------#
        norm_inputs = tf.math.reduce_sum((input*input)) #WLOG if norm is squared!
        
        #------------------------------------------------------------------------------------#
        # Bump Function
        #------------------------------------------------------------------------------------#
#         bump_input = self.a *norm_inputs + self.b
#         greater = tf.math.greater(bump_input, -self.sigma)
#         less = tf.math.less(bump_input, self.sigma)
#         condition = tf.logical_and(greater, less)

#         output_bump = tf.where(
#             condition, 
#             self.bump_function(bump_input),
#             0.0)
        
        
        
        
        #------------------------------------------------------------------------------------#
        # Tangential Map
        #------------------------------------------------------------------------------------#
        # Build Radial, Tangent-Space Valued Function, i.e.: C(R^d,so_d) st. f(x)=f(y) if |x|=|y|
        
        
        # Build Tangential Feed-Forward Network (Bonus)
        #-----------------------------------------------#
        tangential_ffNN = norm_inputs*self.Tw1
        tangential_ffNN = tangential_ffNN + self.Tb1
        tangential_ffNN = tf.nn.relu(tangential_ffNN)  
        tangential_ffNN = tf.matmul(tangential_ffNN, self.Tw2) 
    
        # Scale DNN by Radial-Data
#         tangential_ffNN = output_bump*tangential_ffNN
        # Map to Rotation-Matrix-Valued Function #
        #----------------------------------------#
        tangential_ffNN = (tf.transpose(tangential_ffNN) - tangential_ffNN) 
#         tangential_ffNN = output_bump*tangential_ffNN
        tangential_ffNN = tangential_ffNN + self.num_stab_param*tf.linalg.diag(tf.ones(d+D))
        tangential_ffNN = tf.linalg.expm(tangential_ffNN)
        
        # Exponentiation and Action
        #----------------------------#
        x_out = tf.linalg.matvec(tangential_ffNN,(input-self.location)) + self.location
        
        # Return Output
        return x_out

### Projection Layer

In [5]:
projection_layer = tf.keras.layers.Lambda(lambda x: x[:, -D:])

# Build

In [6]:
# define and fit the base model
def get_base_model(trainx, trainy, Pre_Epochs_in):
    # Define Model
    #----------------#
    # Initialize
    input_layer = tf.keras.Input(shape=[d+D])
    # Apply Reconfiguration Unit
    reconfiguration_unit  = Reconfiguration_unit(d+D)(input_layer)
    # Output
    output_layer = projection_layer(reconfiguration_unit)
    reconfiguration_basic = tf.keras.Model(inputs=[input_layer], outputs=[output_layer])
    
    # Compile Model
    #----------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-2), momentum=0.01, nesterov=True)
    # Compile
    reconfiguration_basic.compile(loss = 'mse',
                    optimizer = optimizer_on,
                    metrics = ['mse'])
    
    # Fit Model
    #----------------#
    reconfiguration_basic.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)
        
    # Return Output
    return reconfiguration_basic

#### Greedy Initialization of Subsequent Units
Build reconfiguration and pre-train using greedy approach.

In [7]:
def add_reconfiguration_unit_greedily(model, trainx, trainy, Pre_Epochs_in):

    # Dissasemble Network
    layers = [l for l in model.layers]

    # Define new reconfiguration unit to be added
    new_reconfiguration_unit  = Reconfiguration_unit(d+D)(layers[len(layers)-2].output)

    # Output Layer
    output_layer_new = projection_layer(new_reconfiguration_unit)

    for i in range(len(layers)):
        layers[i].trainable = False


    # build model
    new_model = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()


    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-2), momentum=0.01, nesterov=True)
    # Compile Model
    new_model.compile(loss = 'mse',
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    new_model.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)

    # Return Output
    return new_model

#### Train and Compile (entire) reconfiguration using greedy-initializations past from previous helper functions.
Train reconfiguration together (initialized by greedy) layer-wise initializations.

In [8]:
def build_reconfiguration(model_greedy_initialized, trainx, trainy, Full_Epochs_in):

    # Dissasemble Network
    layers = [l for l in model_greedy_initialized.layers]

    # Define new reconfiguration unit to be added
    new_reconfiguration_unit  = Reconfiguration_unit(d+D)(layers[len(layers)-2].output)

    # Output Layer
    output_layer_new = projection_layer(new_reconfiguration_unit)

    for i in range(len(layers)):
        layers[i].trainable = True


    # build model
    reconfiguration = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()



    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-5), momentum=0.01, nesterov=True)
    #optimizer_on = tf.keras.optimizers.Adagrad(learning_rate=10**(-5), initial_accumulator_value=0.1, epsilon=1e-07,name='Adagrad')

    # Compile Model
    reconfiguration.compile(loss = 'mse',
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    reconfiguration.fit(trainx, trainy, epochs=Full_Epochs_in, verbose=1)

    # Return Output
    return reconfiguration

# Train the model

In [9]:
# Base Model
model = get_base_model(data_NEU,NEU_targets,10)

# Greedy Initialization
NEU_OLS_Greedy_init = model
for i in range(N_Reconfigurations):
    # Update Model
    NEU_OLS_Greedy_init_temp = add_reconfiguration_unit_greedily(NEU_OLS_Greedy_init,data_NEU,NEU_targets,Pre_Epochs)
    
    # Check for Blowup
    if math.isnan(np.mean(NEU_OLS_Greedy_init.predict(data_NEU))):
        NEU_OLS_Greedy_init = NEU_OLS_Greedy_init
        break
    else: #Update Model if not explosion
        NEU_OLS_Greedy_init = NEU_OLS_Greedy_init_temp
    
    print(np.mean((NEU_OLS_Greedy_init.predict(data_NEU) - data_y)**2))
    
    # Update User on Status of Initialization
    print(((i+1)/N_Reconfigurations))

0.14849957705276223
0.01
0.1474684911725107
0.02
0.145276226610533
0.03
0.14514204969781352
0.04
0.1341321466303362
0.05
0.13539246082980058
0.06
0.13638047131206965
0.07
0.14168174337396233
0.08
0.14205115211298192
0.09
0.14292098444836188
0.1
0.14164868525281055
0.11
0.1405861728060179
0.12
0.14049215001334567
0.13
0.14049215025834308
0.14
0.14049215023898132
0.15
0.14053779533346386
0.16
0.13268387248788643
0.17
0.13268387253909017
0.18
0.13108681260827637
0.19
0.13119582839035945
0.2
0.1309289125491151
0.21
0.11618801929522243
0.22
0.11671784923959255
0.23
0.11809239125846344
0.24
0.11472910050827596
0.25
0.11464519730055175
0.26
0.11413043246635564
0.27
0.12260680623717726
0.28
0.12396668052684393
0.29
0.12406020741752397
0.3
0.1238549678839971
0.31
0.12268117816842565
0.32
0.12262906824421926
0.33
0.12688228743042912
0.34
0.12675776679669196
0.35
0.12691454163838214
0.36
0.12683159132385405
0.37
0.12668158260966866
0.38
0.12628237280687604
0.39


KeyboardInterrupt: 

In [10]:
# Report Train-set Quality
#-------------------------#
print(np.mean(np.abs(model.predict(data_NEU) - NEU_targets)))

0.37338257347819154


## Train NEU-OLS

In [ ]:
# # Train Full Model Using Initializatoins
NEU_OLS = NEU_OLS_Greedy_init
NEU_OLS = build_reconfiguration(NEU_OLS_Greedy_init,data_NEU,NEU_targets,Full_Epochs)

Train on 1000 samples
Epoch 1/100


## Make Predictions

In [ ]:
def above_percentile(x, p): #assuming the input is flattened: (n,)

    samples = Kb.cast(Kb.shape(x)[0], Kb.floatx()) #batch size
    p =  (100. - p)/100.  #100% will return 0 elements, 0% will return all elements

    #samples to get:
    samples = Kb.cast(tf.math.floor(p * samples), 'int32')
        #you can choose tf.math.ceil above, it depends on whether you want to
        #include or exclude one element. Suppose you you want 33% top,
        #but it's only possible to get exactly 30% or 40% top:
        #floor will get 30% top and ceil will get 40% top.
        #(exact matches included in both cases)

    #selected samples
    values, indices = tf.math.top_k(x, samples)

    return values

def Robust_MSE(p):
    def loss(y_true, y_predicted):
        ses = Kb.pow(y_true-y_predicted,2)
        above = above_percentile(Kb.flatten(ses), p)
        return Kb.mean(above)
    return loss

In [ ]:
# # Predictions (for comparison: TEMP)
NEU_OLS_prediction = NEU_OLS(data_NEU)
NEU_OLS_single_unit_prediction = model.predict(data_NEU)
# NEU_OLS_greedy_initializations = NEU_OLS_Greedy_init.predict(data_NEU)

# Visualize Predictions

In [ ]:
# Adjust Figure Details
plt.figure(num=None, figsize=(12, 10), dpi=80, facecolor='w', edgecolor='k')

# Data Plot
plt.plot(data_x,true_y,color='k',label='true',linestyle='--')

# Plot Models
plt.plot(data_x,model_pred_y,color='r',label='OLS')
plt.plot(data_x,NEU_OLS_single_unit_prediction,color='b',label='NEU_Unit')
# plt.plot(data_x,NEU_OLS_greedy_initializations,color='g',label='NEU_Greedy_Init')
plt.plot(data_x,NEU_OLS_prediction,color='Aqua',label='NEU-OLS')

#### The END